<a href="https://colab.research.google.com/github/pandas1003-cyber/eth_btc_price_forecaster/blob/masters/huggingface%E3%81%AEtransformers%E4%BD%BF%E3%81%A3%E3%81%A6%E6%97%A5%E6%9C%AC%E8%AA%9EBert%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#概要
huggingfaceのtransformers使って日本語Bertを試してみる

##必要なライブラリーをダウンロードする
まずは、transformersをインストール前に、形態素解析などのライブラリーをダウンロードする


In [ ]:
#形態素など日本語の自然言語に必要なライブラリーをダウンロードしておく

#参考サイト
#https://qiita.com/Fulltea/items/90f6ebe6dcceaf64eaef
!apt install aptitude swig > /dev/null 
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 
!pip install mecab-python3 > /dev/null 
!pip install fugashi > /dev/null 
!pip install  ipadic > /dev/null 



In [ ]:
#huggingfaceからtransformersをダウンロード、インストールする
!pip install transformers > /dev/null 

In [ ]:
#動作したときのバージョンを記載する
!pip list | grep -e mecab-python3 -e  fugashi -e ipadic -e transformers

fugashi                       1.1.2
ipadic                        1.0.0
mecab-python3                 1.0.5
transformers                  4.17.0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

#日本語の事前学習モデルを読み込む
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

model = AutoModelWithLMHead.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
model.eval()

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:882: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
text = '私は、アメリカで働きたいです'
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

#ここでは「火星」をMASKして、「火星」以外の単語を探そうとしている
masked_index = 3
tokenized_text[masked_index] = '[MASK]'

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)

# Predict
with torch.no_grad():
    outputs = model(tokens_tensor)
    #データ構成 tapleの1個目固定、 [1, 14, 32000] 1個の文章、14つの単語、32000次元の数値
    predictions = outputs[0][0, masked_index].topk(7) # 予測結果の上位7件を抽出

print(outputs[0].shape)
print(predictions.indices)

#「火星」と置き換える文字候補を出す
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()

    #数値を元の単語に戻す
    token = tokenizer.convert_ids_to_tokens([index ])[0]
    print(i, token)

    tokenized_text[masked_index] = token
    print("".join(tokenized_text))

#たしかにそれらしい文章が出力されました。



['私', 'は', '、', 'アメリカ', 'で', '働き', 'たい', 'です']
[1325, 9, 6, 4, 12, 4791, 1549, 2992]
tensor([[1325,    9,    6,    4,   12, 4791, 1549, 2992]])
torch.Size([1, 8, 32000])
tensor([  91,  780, 1411, 1221,  286,  324, 2249])
0 日本
私は、日本で働きたいです
1 中国
私は、中国で働きたいです
2 ここ
私は、ここで働きたいです
3 そこ
私は、そこで働きたいです
4 アメリカ
私は、アメリカで働きたいです
5 世界
私は、世界で働きたいです
6 ロンドン
私は、ロンドンで働きたいです
